# M5 Forecasting Competition - Accuracy
## Step 1. Data Wrangling

This project is based on the accuracy component of the M5 Forecasting competition hosted on www.kaggle.com and run by MOFC, https://mofc.unic.ac.cy/m5-competition/. The competition provided:

1. Hierarchical time series data for sales of items at Walmart stores in three states
2. Time series data for the price of each item
3. A calendar that included dates for promotional events and holidays

The task for the accuracy component of the competition was to predict the sales volume of each item on each day for the subsequent 28 days.

### 1. Load Data

The data is provided in three CSV files, which I have compressed in order to fit under the size limit on GitHub. These files can be read into pandas dataframes for preprocessing. 

In [1]:
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

train = './data/sales_train_evaluation.gz'
calendar = './data/calendar.gz'
sell_prices = './data/sell_prices.gz'

train = pd.read_csv(train, index_col = 0, compression = 'gzip')
calendar =  pd.read_csv(calendar, index_col = 0, compression = 'gzip')
sell_prices =  pd.read_csv(sell_prices, index_col = 0, compression = 'gzip')

The train data is currently in a **wide** format with one row for each of 30490 time series. Each series has a unique **id** followed by five columns that separate the hierarchical identifying information:

1. State (3 total: CA, TX, WI)
2. Store (10 total: CA_1 to CA_4; TX_1 to TX_3; WI_1 to WI_3)
3. Category (3 total: HOBBIES, FOODS, HOUSEHOLD)
4. Department (7 total: HOBBIES_1 and HOBBIES_2; FOODS_1 to FOODS_3; HOUSEHOLD_1 and HOUSEHOLD_2)
5. Item (3049 total)

This is followed by sales data for 1941 days spanning 2011-01-29 to 2016-06-19 (roughly 5.4 years). 

In [2]:
train.shape

(30490, 1947)

In [3]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


The price data is in a **tall** format with the sell price keyed to the item (identified by **store_id** + **item_id**) and an integer identifying the week (**wm_yr_wk**).

In [4]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


Finally, the calendar is also in a **tall** format with one record for each date of the time series. The calender provides the **wm_yr_wk** id that will allow the price data to be joined to the sales data as well as further information about the day, such as events that occur on that day and/or promotions (snap days). 

In [5]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


### 2. Transform sales data to tall format

Before we can fit and evaluate forecasting models, the sales data (train) must be transformed from the **wide** format to a **tall** format with one row per observation (sales volume for each item at each store on a single day). We can pivot the dataframe using the **melt** method provided by pandas.  

In [6]:
ids = ['id', 'item_id', 'dept_id','cat_id','store_id','state_id'] ## The id columns will be maintained 
sales = pd.melt(train, id_vars=ids, var_name = 'd',value_name = 'units')
del train # clear memory as we go 

The melted table now has a whopping 59,181,090 observations!

In [7]:
sales.shape

(59181090, 8)

In [8]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,units
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


### 3. Merge sales data with pricing and calendar

The item prices for each week as well as further information about each day in the time series are contained in the other two dataframes. These can be combined with the sales data using the **merge** method provided by pandas, which performs a database style join. Note that merge performs an inner join by default. For this application, we need to specify a left join in order to every row from the sales data.

The calendar data must be merged with sales first since it contains the **wm_yr_wk** identifier that is used by the sales data.

In [9]:
sales_calendar = pd.merge(sales, calendar, how = 'left', on = 'd')
del sales
del calendar

The merge adds 13 columns from the calendar data.

In [10]:
sales_calendar.shape

(59181090, 21)

In [11]:
sales_calendar.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,units,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0


The price data can then be merged based on the columns **store_id**, **item_id**, and **wm_yr_wk**.

In [12]:
sales_calendar_prices = pd.merge(sales_calendar, sell_prices, how = 'left', on=['store_id', 'item_id', 'wm_yr_wk'])
del sales_calendar
del sell_prices

In [13]:
sales_calendar_prices.shape

(59181090, 22)

In [14]:
sales_calendar_prices.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,units,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


The final data frame includes the target variable **units** along with 20 features on which to base the forecast. These include 6 identifying features, 13 features related to the calendar, and the sale price.